In [1]:
import numpy as np
import pandas as pd
import pickle
from collections import defaultdict
import re
from bs4 import BeautifulSoup
import sys
import os
os.environ['KERAS_BACKEND']='theano'
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional
from keras.models import Model
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
plt.switch_backend('agg')
from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers
%matplotlib inline

Using Theano backend.


In [2]:
def clean_str(string):
    string = re.sub(r"\\", "", string)
    string = re.sub(r"\'", "", string)
    string = re.sub(r"\"", "", string)
    return string.strip().lower()

In [3]:

MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [4]:
# reading data
df = pd.read_csv('./data/train.tsv', sep='\t', header=0)
df = df.dropna()
df = df.reset_index(drop=True)
print('Shape of dataset ',df.shape)
print(df.columns)
print('No. of unique classes',len(set(df['Sentiment'])))

('Shape of dataset ', (156060, 4))
Index([u'PhraseId', u'SentenceId', u'Phrase', u'Sentiment'], dtype='object')
('No. of unique classes', 5)


In [5]:
macronum=sorted(set(df['Sentiment']))
macro_to_id = dict((note, number) for number, note in enumerate(macronum))

def fun(i):
    return macro_to_id[i]

df['Sentiment']=df['Sentiment'].apply(fun)

In [6]:
texts = []
labels = []


for idx in range(df.Phrase.shape[0]):
    text = df.Phrase[idx]
    texts.append(clean_str(text))

for idx in df['Sentiment']:
    labels.append(idx)

In [7]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index

print('Number of Unique Tokens',len(word_index))

('Number of Unique Tokens', 15276)


In [8]:
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


labels = to_categorical(np.asarray(labels))
print('Shape of Data Tensor:', data.shape)
print('Shape of Label Tensor:', labels.shape)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]


('Shape of Data Tensor:', (156060, 100))
('Shape of Label Tensor:', (156060, 5))


In [9]:
embeddings_index = {}
GLOVE_LOC = '../glove.6B/glove.6B.100d.txt'
f = open(GLOVE_LOC)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [10]:
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [11]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [12]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm = Bidirectional(LSTM(100))(embedded_sequences)
preds = Dense(len(macronum), activation='softmax')(l_lstm)
model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

print("Bidirectional LSTM")
model.summary()

Bidirectional LSTM
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          1527700   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1005      
Total params: 1,689,505
Trainable params: 1,689,505
Non-trainable params: 0
_________________________________________________________________


In [13]:
cp=ModelCheckpoint('model_rnn_1.hdf5',monitor='val_acc',verbose=1,save_best_only=True)


In [ ]:
history=model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=10, batch_size=2048,callbacks=[cp])

Train on 124848 samples, validate on 31212 samples
Epoch 1/10
124848/124848 [==============================] - 10413s 83ms/step - loss: 1.1551 - acc: 0.5333 - val_loss: 1.0438 - val_acc: 0.5704

Epoch 00001: val_acc improved from -inf to 0.57036, saving model to model_rnn_1.hdf5
Epoch 2/10
124848/124848 [==============================] - 8791s 70ms/step - loss: 0.9857 - acc: 0.5950 - val_loss: 0.9796 - val_acc: 0.5969

Epoch 00002: val_acc improved from 0.57036 to 0.59692, saving model to model_rnn_1.hdf5
Epoch 3/10
124848/124848 [==============================] - 7552s 60ms/step - loss: 0.9048 - acc: 0.6259 - val_loss: 0.8962 - val_acc: 0.6287

Epoch 00003: val_acc improved from 0.59692 to 0.62870, saving model to model_rnn_1.hdf5
Epoch 4/10
124848/124848 [==============================] - 9006s 72ms/step - loss: 0.8541 - acc: 0.6475 - val_loss: 0.8732 - val_acc: 0.6399

Epoch 00004: val_acc improved from 0.62870 to 0.63985, saving model to model_rnn_1.hdf5
Epoch 5/10
124848/124848 [=

fig1 = plt.figure()
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves :RNN',fontsize=16)
fig1.savefig('loss_rnn.png')
plt.show()

fig2=plt.figure()
plt.plot(history.history['acc'],'r',linewidth=3.0)
plt.plot(history.history['val_acc'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves : RNN',fontsize=16)
fig2.savefig('accuracy_rnn.png')
plt.show()

#from keras.utils.vis_utils import plot_model
#plot_model(model, to_file='rnn_model.png', show_shapes=True, show_layer_names=True)

from PIL import Image
display(Image.open('rnn_model.png'))